In [1]:
import pyupbit
import time
import json
import requests
import pandas as pd
import mysql.connector

##### DB연결

In [43]:
# MySQL 서버에 연결
mydb = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="1234",
    database="deepCoin"
)

# 연결이 제대로 되었는지 확인
if mydb.is_connected():
    print("MySQL 데이터베이스에 성공적으로 연결되었습니다.")
else:
    print("MySQL 데이터베이스 연결에 실패했습니다.")

# 커서 생성
mycursor = mydb.cursor()


MySQL 데이터베이스에 성공적으로 연결되었습니다.


##### DB 종목마스터 가져오기

In [50]:
# SELECT 쿼리 실행
mycursor.execute("SELECT * FROM c_krw_mst")

# 결과 가져오기
result = mycursor.fetchall()

# 튜플의 0번째 인덱스만 가져와서 마켓 리스트 생성
market_list = [t[0] for t in result]

In [58]:
market_list = ['KRW-ADA','KRW-ADA']

In [81]:
market = 'KRW-ADA'
end_date = '20240101'
select_count = 200
# for market in market_list:
url = f"https://api.upbit.com/v1/candles/days?market={market}&to={end_date}&count={select_count}&convertingPriceUnit=KRW"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)

mst_json = json.loads(response.text)

In [82]:
mst_json

{'error': {'name': 400,
  'message': 'Invalid parameter. Check the given value!'}}

In [39]:
url = "https://api.upbit.com/v1/candles/days?market=KRW-BTC&count=200&convertingPriceUnit=KRW"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)

mst_json = json.loads(response.text)

In [40]:
for i in 
url = "https://api.upbit.com/v1/candles/days?market=KRW-BTC&count=200&convertingPriceUnit=KRW"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)

mst_json = json.loads(response.text)

##### 종목마스터 수집

In [25]:
url = "https://api.upbit.com/v1/market/all?isDetails=true"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)

mst_json = json.loads(response.text)
df = pd.DataFrame(mst_json)
df = df[['market', 'korean_name', 'english_name', 'market_warning']]
df = df[df['market'].str.contains('KRW')].reset_index(drop=True)


# DB생성용 Master 항목당 최대 길이 출력
max_length = df['market'].str.len().max()
max_string = df[df['market'].str.len() == max_length]['market'].iloc[0]
# print(max_string, len(max_string))

# DataFrame을 튜플의 리스트로 변환
values = df.values.tolist()

# 컬럼명 리스트
columns = df.columns.tolist()

# 삽입할 테이블 이름
table_name = "c_krw_mst"

In [26]:
# 데이터 삽입 쿼리 생성
sql = "INSERT INTO {} ({}) VALUES ({})".format(
    table_name,
    ', '.join(columns),
    ', '.join(['%s'] * len(columns))
)

# 데이터 삽입
mycursor.executemany(sql, values)

# 변경 사항 커밋
mydb.commit()
print(mycursor.rowcount, "레코드가 삽입되었습니다.")
mydb.close()

118 레코드가 삽입되었습니다.
